In [13]:
import numpy as np
from sklearn.neural_network import MLPClassifier as MLPNC
from sklearn.externals import joblib as joblib
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score

In [7]:
train = np.loadtxt('../datosEntrenamiento.csv', delimiter=',')
test = np.loadtxt('../datosPrueba.csv', delimiter=',')
train_x = train[:,:-1]
train_y = train[:, train.shape[1]-1]
test_x = test[:,:-1]
test_y = test[:, test.shape[1]-1]

# scale data into [0,1)
scaler = MinMaxScaler().fit(train_x) 
train_x = scaler.transform(train_x)
test_x = scaler.transform(test_x)

le = preprocessing.LabelEncoder()
le.fit(train_y)
train_y = le.transform(train_y) # needed for MLPNClassifier
test_y = le.transform(test_y)

print(train_x[:5,:5])

[[ 0.84126984  0.8         0.77011494  0.50413223  0.69230769]
 [ 0.71428571  0.68181818  0.56321839  0.41322314  0.63076923]
 [ 0.71428571  0.60909091  0.56321839  0.38016529  0.63076923]
 [ 0.65079365  0.60909091  0.51724138  0.3553719   0.63076923]
 [ 0.58730159  0.68181818  0.6091954   0.41322314  0.56923077]]


In [17]:

alphas = [.01,.05,.001,.005,.0001,.0005]
max_score = 0
min_var = 1
max_score_alpha = alphas[0]
for a in alphas:    
    mlpn = MLPNC(solver='lbfgs', alpha=a, hidden_layer_sizes=(100,))
    scores = cross_val_score(mlpn, train_x, train_y, cv=10) # 10 fold crossvalidation
    m_score = scores.mean()
    var_score = scores.std() * 2
    if max_score < m_score and min_var > var_score:
        max_score = m_score
        min_var = var_score
        mas_score_alpha = a
    print("Accuracy(with alpha = %f): %0.2f (+/- %0.2f)" % (a, m_score, var_score))
    
mlpn = MLPNC(solver='lbfgs', alpha=max_score_alpha, hidden_layer_sizes=(100,))
mlpn.fit(train_x, train_y)
accuracy = mlpn.score(test_x, test_y)
print('Model test accuracy: %0.3f'%(accuracy))

Accuracy(with alpha = 0.010000): 0.86 (+/- 0.08)
Accuracy(with alpha = 0.050000): 0.86 (+/- 0.09)
Accuracy(with alpha = 0.001000): 0.87 (+/- 0.08)
Accuracy(with alpha = 0.005000): 0.87 (+/- 0.08)
Accuracy(with alpha = 0.000100): 0.87 (+/- 0.08)
Accuracy(with alpha = 0.000500): 0.87 (+/- 0.07)
Model test accuracy: 0.901


In [18]:
joblib.dump(mlpn, 'models/MLPC.pkl') 

['models/MLPC.pkl']

In [23]:
validation = np.loadtxt('../datosValidacion.csv',delimiter=',')
validation = scaler.transform(validation)
validation_output = mlpn.predict(validation)
print(validation_output[:10])
np.savetxt('../retoFiltro.txt', validation_output, fmt='%d')

[ 1.  1.  2.  1.  1.  1.  1.  1.  1.  4.]
